User-based collaborative filtering. This technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. <br>

I am going to use simple method here based on the Pearson Correlation using pandas. It's a simple approach and it doesn't cover many topics, but it good for get acquainted with recommender systems.

In [1]:
import numpy as np
import pandas as pd

In [2]:
books_df = pd.read_csv('books.csv')
ratings_df = pd.read_csv('ratings.csv')

In [3]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   book_id                    10000 non-null  int64  
 1   goodreads_book_id          10000 non-null  int64  
 2   best_book_id               10000 non-null  int64  
 3   work_id                    10000 non-null  int64  
 4   books_count                10000 non-null  int64  
 5   isbn                       9300 non-null   object 
 6   isbn13                     9415 non-null   float64
 7   authors                    10000 non-null  object 
 8   original_publication_year  9979 non-null   float64
 9   original_title             9415 non-null   object 
 10  title                      10000 non-null  object 
 11  language_code              8916 non-null   object 
 12  average_rating             10000 non-null  float64
 13  ratings_count              10000 non-null  int6

In [4]:
books_df.sample(5)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
2097,2098,9590,33290408,4076,80,385333471,9.780385e+12,Kurt Vonnegut Jr.,1965.0,"God Bless You, Mr Rosewater or Pearls before S...",...,40353,45412,1444,203,1670,11416,19313,12810,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
1388,1389,22284,22284,1602261,59,1400032814,9.781400e+12,Chuck Palahniuk,2003.0,Diary,...,58428,61369,2196,1633,6299,19618,21016,12803,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
8559,8560,19049,19049,106848,16,075730334X,9.780757e+12,"William Arntz, Betsy Chasse, Mark Vicente",2005.0,What the Bleep Do We Know!?: Discovering the E...,...,9759,10033,142,212,369,1387,2820,5245,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
6862,6863,218659,218659,963076,37,374504644,9.780375e+12,"Flannery O'Connor, Robert Fitzgerald",1965.0,Everything that Rises Must Converge,...,14534,15025,873,163,433,1876,5312,7241,https://images.gr-assets.com/books/1412859621m...,https://images.gr-assets.com/books/1412859621s...
6703,6704,139569,139569,1523329,14,1555838537,9.781556e+12,Leslie Feinberg,1993.0,Stone Butch Blues,...,13570,13969,760,197,411,1948,4256,7157,https://images.gr-assets.com/books/1328758827m...,https://images.gr-assets.com/books/1328758827s...


In [5]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5976479 entries, 0 to 5976478
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   user_id  int64
 1   book_id  int64
 2   rating   int64
dtypes: int64(3)
memory usage: 136.8 MB


In [6]:
ratings_df.sample(5)

,user_id,book_id,rating
5921342,46677,7487,5
1197210,17830,3,1
1403815,20282,2849,4
1888927,26129,6494,5
5645005,29265,110,5


As a fan of literature I prepared my own list of some my favourite books (you can also use any user from dataset): 

   - Jack London _"Martin Eden"_ <br>
   - Franz Kafka _"The trial"_ <br>
   - Stephen King _"Pet Sematary"_ <br>
   - Gabriel Garcia Markes _"One Hundred Years of Solitude"_ <br>
   - Charles Bukowski _"Ham on Rye"_ <br>
   - John Steinbeck _"The Grapes of Wrath"_ <br>
   - Kurt Vonnegut _"Cat's Cradle"_ <br>
   - Fedor Dostoyevsky _"Crime and Punishment"_ <br>

And below I rated these books.

In [7]:
my_list = {'Martin Eden': 5,
            'Pet Sematary': 5,
            'One Hundred Years of Solitude': 5,
            'Ham on Rye': 5,
            'The Grapes of Wrath': 4, 
            "Cat's Cradle": 5,
            'Crime and Punishment': 4,
            'The Trial': 4}

In [8]:
# Create dataframe for new user (me)
user_books = pd.DataFrame(columns=['title', 'rating'], data=my_list.items())

# Add book_id from books_df
new_user = pd.merge(user_books, books_df, on='title', how='inner')
new_user = new_user[['book_id', 'title', 'rating']].sort_values(by='book_id')
new_user

,book_id,title,rating
2,94,One Hundred Years of Solitude,5
4,131,The Grapes of Wrath,4
6,177,Crime and Punishment,4
5,297,Cat's Cradle,5
1,305,Pet Sematary,5
7,614,The Trial,4
3,1444,Ham on Rye,5
0,7068,Martin Eden,5


Select a users with books I have read.

In [9]:
other_users = ratings_df[ratings_df['book_id'].isin(new_user['book_id'].values)]
other_users

,user_id,book_id,rating
38,4,297,4
49,8,614,4
160,11,94,5
182,8,177,5
276,4,131,4
...,...,...,...
5973753,39833,94,1
5973913,48436,297,4
5974295,51542,94,4
5974628,7607,94,5


In [10]:
other_users['user_id'].nunique()

18637

18363 users had read at least one book from my list. 

In [11]:
# Sort users by count of most mutual books with me
users_mutual_books = other_users.groupby(['user_id'])
users_mutual_books = sorted(users_mutual_books, key=lambda x: len(x[1]), reverse=True)
users_mutual_books[0]

(47897,
          user_id  book_id  rating
 4378842    47897      177       5
 4961030    47897      131       5
 4961036    47897     1444       5
 4961076    47897       94       5
 4961530    47897      297       5
 4961537    47897     7068       5
 4961543    47897      614       4)

I will be using only first 100 records, these with the most common part of books

In [12]:
top_users = users_mutual_books[:100]

Now let's calculate a similarity score for each user using Pearson correlation function and use these scores as weights later. 

In [13]:
# Pearson correlation
from scipy.stats import pearsonr

pearson_corr = {}

for name, books in top_users:
    # Books should be sorted
    books = books.sort_values(by='book_id')
    
    book_list = sorted(books['book_id'].values)

    user_ratings = new_user[new_user['book_id'].isin(book_list)]['rating'].values 
    group_ratings = books[books['book_id'].isin(book_list)]['rating'].values
   
    corr = pearsonr(user_ratings, group_ratings)
    pearson_corr[name] = corr[0]

/Users/mark/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [14]:
# Get top50 users with the highest similarity indices
pearson_df = pd.DataFrame(columns=['user_id', 'similarity_index'], data=pearson_corr.items())
pearson_df = pearson_df.sort_values(by='similarity_index', ascending=False)[:50]
pearson_df

,user_id,similarity_index
33,11862,1.000000
31,11274,0.912871
57,27747,0.912871
32,11356,0.872872
20,5426,0.866025
9,1027,0.763763
43,17944,0.763763
50,21646,0.745356
26,8616,0.720577
1,10898,0.707107


In [15]:
# Get all books for these users and add weighted book's ratings
users_rating = pearson_df.merge(ratings_df, on='user_id', how='inner')
users_rating['weighted_rating'] = users_rating['rating'] * users_rating['similarity_index']
users_rating

,user_id,similarity_index,book_id,rating,weighted_rating
0,11862,1.000000,4,4,4.000000
1,11862,1.000000,8,4,4.000000
2,11862,1.000000,5,3,3.000000
3,11862,1.000000,14,3,3.000000
4,11862,1.000000,50,3,3.000000
...,...,...,...,...,...
6752,32921,0.080064,223,3,0.240192
6753,32921,0.080064,3418,1,0.080064
6754,32921,0.080064,184,4,0.320256
6755,32921,0.080064,7879,3,0.240192


Now it rests only calculate average recommendation score and select items with the highest score. <br>

In [16]:
# Calculate sum of similarity index and weighted rating for each book
grouped_ratings = users_rating.groupby('book_id').sum()[['similarity_index', 'weighted_rating']]

In [17]:
recommend_books = pd.DataFrame()

recommend_books['avg_reccomend_score'] = grouped_ratings['weighted_rating']/grouped_ratings['similarity_index']
recommend_books['book_id'] = grouped_ratings.index

recommend_books = recommend_books.reset_index(drop=True)

# Left books with the highest score
recommend_books = recommend_books[recommend_books['avg_reccomend_score'] == 5]

recommend_books

,avg_reccomend_score,book_id
129,5.0,145
130,5.0,147
133,5.0,151
157,5.0,185
196,5.0,245
...,...,...
1887,5.0,9463
1891,5.0,9561
1892,5.0,9605
1898,5.0,9722


Let's have a look for list of 10 recommendation. 

In [18]:
recommendation = books_df[books_df['book_id'].isin(recommend_books['book_id'])][['authors', 'title', 'book_id']].sample(10)
recommendation


,authors,title,book_id
2943,Terry Pratchett,"The Truth (Discworld, #25; Industrial Revoluti...",2944
5201,David Mazzucchelli,Asterios Polyp,5202
1630,"Arthur Conan Doyle, Peter Ackroyd, Ed Glinert",The Sign of Four,1631
2451,Terry Pratchett,"Pyramids (Discworld, #7)",2452
3023,Edward Abbey,Desert Solitaire,3024
640,C.S. Lewis,"The Last Battle (Chronicles of Narnia, #7)",641
3450,"Rainer Maria Rilke, Reginald Snell, Franz Xave...",Letters to a Young Poet,3451
3463,Aldous Huxley,The Doors of Perception & Heaven and Hell,3464
579,E.H. Gombrich,The Story of Art,580
1297,Jonathan Stroud,"The Golem's Eye (Bartimaeus, #2)",1298
